In [ ]:
import pandas as pd
import csv
import datetime
import os
import statistics
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import math

In [ ]:
# Getcha dat data.  You could theoretically use any kind of log that includes a timestamp.  
# For this instance, I'm using an export CSV of outbound http data.

df = pd.read_csv('C:\\path\\to\\file.csv', header=0, parse_dates=["@timestamp"])
df.dropna(inplace=True)
df.sort_values(by='@timestamp', ascending='True')



In [ ]:
# Start with just a simple timeseries histogram, to see count of traffic over time
fig=px.histogram(df, x='@timestamp')
fig.show()

In [ ]:
# Set custom time lengths for a sampling spacing.  Try a bunch, so we can see what we get.
# The sample spacing is in terms of seconds per sample.  Its inverse is the sample rate, 
# in samples per second.  

#Express in terms of seconds.
sampleperiods = ['1s', '10s', '20s', '30s', '60s', '120s', '300s', '600s', '1200s', '1800s', '3600s'] 

 # The sampling rate must be at least 2* the highest frequency we're trying to find.
 # The sampling period is 1/(sample rate).
 # Therefore, the sample period must be AT MOST 1/2 the period you are trying to find.  
 # For example, to find periodic activity with a period of 1 minute (frequency=0.0167 Hz),
 # you will need a sample period of no higher than 30 sec (frequency=0.03 Hz).
 #
 # The sample periods above can find periodicities of up to 2 hours (7200 sec).


In [ ]:
# Re-index the timestamps, so we can get counts per timeslice.
tmpdata = df['@timestamp']
tmpdata.index = tmpdata

In [ ]:
# Apply a Fast Fourier Transform to the numeric counts per time slice, and plot it
for period in sampleperiods:
    # First, get the count of http events per time slice
    countsperperiod = tmpdata.resample(period).count()
    
    # Apply a Real-Input Fast Fourier Transform on the counts of items per timeslice.  This gets the
    # signal strength (amplitude) of each present frequency.  This will become the yaxis.
    fft = np.fft.rfft(countsperperiod)  
    
    # turn the current sampling period into an int
    dvalue = int(period.rstrip("s")) 
    
    # Use the Real-Input Fast Fourier Transform Frequency
    # rfftfreq() function to get the frequencies present in the signal.
    # This will become the xaxis.
    #
    # Arguments to rfftfreq():
    # n = window length, defined as the number of items in the current sample spacing
    # d = sample spacing, which is defined above (also equals 1/samplerate)
    frequencies = np.fft.rfftfreq(len(countsperperiod), dvalue)
    
    # Now, plot the frequency (inverse of period) vs amplitude, in Hertz
    fig = px.line(
        x=frequencies,
        y=(abs(fft)), #fft is a complex number; plotting its absolute value gives the amplitude
        labels=dict(x="Frequency (cycle/sec)", y="HTTP GET Requests"),
        title="HTTP GETs by Frequency; Sampling Period: " + period
    )
    fig.update_yaxes(
        range=(0, 60000),
        constrain='domain'
    )
    fig.show()
    


In [ ]:
# From whatever you found as the best samnple period earlier, that shows the strongest signal spikes.
# Set this to form 'xxs', where 'xx' is the length of the best-fit periodicity
bestperiod = '20s' 

In [ ]:
# Identify the strongest signals present in the frequency graph (spikes). 
# First, let's resample the data and get the FFT and FFTFreq for the best sampling period we found earlier.
countsperperiod = tmpdata.resample(bestperiod).count()
fft = abs(np.fft.rfft(countsperperiod))
dvalue = int(bestperiod.rstrip("s")) 
frequencies = np.fft.rfftfreq(len(countsperperiod), dvalue)
 
# Get any signal spikes over CONST * stdev over the rest of the noise.  This will be the interesting stuff to look
# at.  The amplitudes (y-values) come from the fft array found above.
# Find the standard deviation of the remaining data, so we can use it to find the strongest signals present.  
# Strip off the first 10% of the frequencies found, which will remove the DC component of the signal, leaving you with 
# just the actual signal spikes.
strippedfrequencies = frequencies[frequencies > 0.1*max(frequencies)]
strippedfft = fft[len(fft)-strippedfrequencies.size:]

stdev = np.std(strippedfft)
mean = np.mean(strippedfft)
threshold = mean + 4*stdev  # Adjust that multiplier as needed, higher = less results
print("mean: " + str(mean) + ", stdev: " + str(stdev) + ", threshold: " + str(threshold))

strongsignals = []
for signal in strippedfft:
    if (signal > threshold): 
        print("adding signal: " +str(signal))
        strongsignals.append(signal)
        
# Plot the frequency data after removing the DC component
fig = px.line(
    x=strippedfrequencies,
    y=(abs(strippedfft)),
    labels=dict(x="Frequency (cycle/sec)", y="HTTP GET Requests"),
    title="HTTP GETs by Frequency With DC Removed; Sampling Period: " + bestperiod
)
fig.show()

In [ ]:
# For each strong signal: find the array index from the fft array
signalindices = []
i = 0
while (i < len(strongsignals)):
    matchingindex = np.where(fft == np.float64(strongsignals[i]))[0][0]
    signalindices.append(matchingindex)
    i += 1

# Create a new array of the same size as the fft array.  Zero it out,
# except for the indices you just found, which are the strong signals we
# want to find the times for.
strongsignalfrequencies = np.zeros(len(fft))
for index in signalindices:
    strongsignalfrequencies[index] = frequencies[index]
    
strongsignalamplitudes = np.zeros(len(fft))
for index in signalindices:
    strongsignalamplitudes[index] = fft[index]

In [ ]:
# Graph the data in the time domain, by your chosen sampling period
fig = px.line(
    countsperperiod,
    labels=dict(x="Timestamp", y="HTTP GET Requests"),
    title="HTTP GETs By Timestamp; Sampling Period: " + bestperiod
)
fig.show()

In [ ]:
# De-noise the data by filtering. Make an effective bandpass filter by
# zeroing out all the frequencies except the strong ones found above.
# Plot just the strong signal frequencies vs their amplitudes.
fig = px.line(
    x=frequencies,
    y=(abs(strongsignalamplitudes)),
    labels=dict(x="Frequency (cycle/sec)", y="HTTP GET Requests"),
        title="Strongest Signal Frequencies; Sampling Period: " + bestperiod
)
fig.update_yaxes(
    range=(0, 1.5*max(strongsignalamplitudes)),
    constrain='domain'
)
fig.update_xaxes(
    
)
fig.show()

In [ ]:
# Use the Inverse FFT to flip just the strong signals back to time-domain
inversefft = np.fft.irfft(strongsignalamplitudes, len(countsperperiod))

fig = px.line(
    x=countsperperiod.to_frame().index,
    y=inversefft,
    labels=dict(x="Timestamp", y="Count of HTTP Get Requests"),
        title="Periodic Signal"
)

fig.show()

In [ ]:
# OK.  Now, for each of our strong signals, we need to identify domains from our original data set that 
# had a count of GET requests "near" our signal strengths.  (It won't be spot-on, due to sample frequency 
# bin width and signal jitter.)  This will be the shortlist of domains for further investigation.
shortlist = []
newdf = df.groupby(['host']).size().reset_index(name='counts')
for amplitude in strongsignals:
    shortlist.append(newdf[ (newdf['counts'] > (amplitude*0.8)) & (newdf['counts'] < (amplitude*1.2)) ])
    
results = pd.concat(shortlist, ignore_index=True)
print(results)
results[['host','counts']].to_csv('c:\\hunting\\output.csv')